# Consulting Project 
## Recommender Systems - Solutions

The whole world seems to be hearing about your new amazing abilities to analyze big data and build useful systems for them! You've just taken up a new contract with a new online food delivery company. This company is trying to differentiate itself by recommending new meals to customers based off of other customers likings.

Can you build them a recommendation system?

Your final result should be in the form of a function that can take in a Spark DataFrame of a single customer's ratings for various meals and output their top 3 suggested meals. For example:

Best of luck!

** *Note from Jose: I completely made up this food data, so its likely that the actual recommendations themselves won't make any sense. But you should get a similar output to what I did given the example customer dataframe* **

In [39]:
!pip install findspark


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession


In [11]:
# !pip install pyspark

     ---------------------------------------- 0.0/317.0 MB ? eta -:--:--
     --------------------------------------- 0.5/317.0 MB 16.2 MB/s eta 0:00:20
     --------------------------------------- 2.1/317.0 MB 27.2 MB/s eta 0:00:12
     --------------------------------------- 2.9/317.0 MB 23.5 MB/s eta 0:00:14
     --------------------------------------- 3.5/317.0 MB 18.8 MB/s eta 0:00:17
     --------------------------------------- 3.9/317.0 MB 17.7 MB/s eta 0:00:18
     --------------------------------------- 3.9/317.0 MB 17.7 MB/s eta 0:00:18
     --------------------------------------- 3.9/317.0 MB 17.7 MB/s eta 0:00:18
      -------------------------------------- 4.1/317.0 MB 11.9 MB/s eta 0:00:27
      -------------------------------------- 4.1/317.0 MB 11.9 MB/s eta 0:00:27
      -------------------------------------- 6.4/317.0 MB 14.7 MB/s eta 0:00:22
      -------------------------------------- 7.1/317.0 MB 14.7 MB/s eta 0:00:22
      -------------------------------------- 7.

In [2]:
# pip install pandas

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
altair 5.2.0 requires toolz, which is not installed.
category-encoders 2.6.3 requires patsy>=0.5.1, which is not installed.
category-encoders 2.6.3 requires scipy>=1.0.0, which is not installed.
category-encoders 2.6.3 requires statsmodels>=0.9.0, which is not installed.
sagemaker 2.212.0 requires cloudpickle==2.2.1, which is not installed.
sagemaker 2.212.0 requires importlib-metadata<7.0,>=1.4.0, which is not installed.
sagemaker 2.212.0 requires tblib<3,>=1.7.0, which is not installed.
streamlit 1.32.2 requires pillow<11,>=7.1.0, which is not installed.
streamlit 1.32.2 requires pyarrow>=7.0, which is not installed.
streamlit 1.32.2 requires tenacity<9,>=8.1.0, which is not installed.
streamlit 1.32.2 requires toml<2,>=0.10.1, which is not installed.
streamlit 1.32.2 requires watchdog>=2.1.5; platform_system !=

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.4/11.6 MB 3.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.6 MB 5.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.6 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.6 MB 6.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.6 MB 7.5 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.6 MB 8.2 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/11.6 MB 9.1 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 9.9 MB/s eta 0:00:01
   ----------------- ---------------------- 5.1/11.6 MB 10.9 MB/s eta 0:00:01
   ------------------

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\Gnaneshwari\Downloads\Telegram Desktop\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Recommender_Systems\movielens_ratings.csv")

In [4]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [5]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [6]:
import numpy as np
df['mealskew'] = df['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [7]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


In [8]:
mealmap = { 2. : "Chicken Curry",   
           3. : "Spicy Chicken Nuggest",   
           5. : "Hamburger",   
           9. : "Taco Surprise",  
           11. : "Meatloaf",  
           12. : "Ceaser Salad",  
           15. : "BBQ Ribs",  
           17. : "Sushi Plate",  
           19. : "Cheesesteak Sandwhich",  
           21. : "Lasagna",  
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",  
           27. : "Salmon with Mashed Potatoes",  
           28. : "Penne Tomatoe Pasta",  
           29. : "Pork Sliders",  
           30. : "Vietnamese Sandwich",  
           31. : "Chicken Wrap",  
           np.nan: "Cowboy Burger",   
           4. : "Pretzels and Cheese Plate",   
           6. : "Spicy Pork Sliders",  
           13. : "Mandarin Chicken PLate",  
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",  
           8. : "Chicken Chow Mein",  
           10. : "Roasted Eggplant ",  
           18. : "Pepperoni Pizza",  
           22. : "Pulled Pork Plate",   
           0. : "Cheese Pizza",   
           1. : "Burrito",   
           7. : "Nachos",  
           24. : "Chili",  
           20. : "Southwest Salad",  
           25.: "Roast Beef Sandwich"}

In [9]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [10]:
df.to_csv('Meal_Info.csv',index=False)

In [11]:
# pip install pyspark

In [12]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.appName('recconsulting').getOrCreate()

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [15]:
data = spark.read.csv(r"C:\Users\Gnaneshwari\Downloads\Telegram Desktop\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Recommender_Systems\Meal_Info.csv",inferSchema=True,header=True)

In [16]:
(training, test) = data.randomSplit([0.8, 0.2])

In [17]:
# pip install -c cyclus java-jdk

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [25]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
              coldStartStrategy="drop")
model = als.fit(training)

In [26]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

predictions.show()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+------+------+--------+--------------------+-----------+
|movieId|rating|userId|mealskew|           meal_name| prediction|
+-------+------+------+--------+--------------------+-----------+
|     12|   5.0|    28|    12.0|        Ceaser Salad|0.030542046|
|     13|   2.0|    28|    13.0|Mandarin Chicken ...|  0.8074275|
|     17|   1.0|    28|    17.0|         Sushi Plate|  1.4492674|
|     34|   1.0|    28|    null|                null|  0.9307044|
|     52|   1.0|    28|    null|                null|   3.143499|
|     63|   1.0|    28|    null|                null|   2.444973|
|     78|   1.0|    28|    null|                null|    1.33475|
|     81|   5.0|    28|    null|                null|   1.628796|
|     99|   1.0|    28|    null|                null|  1.2300649|
|      0|   1.0|    27|     0.0|        Cheese Pizza|  1.5279856|
|      5|   2.0|    26|     5.0|           Hamburger|   1.420919|
|     10|   1.0|    27|    10.0|   Roasted Eggplant |  1.8179598|
|     13| 